In [ ]:
from pathlib import Path

import lsdb
import matplotlib.pyplot as plt
import numpy as np
from gatspy.periodic import LombScargleMultiband

In [ ]:
OID = 592913913020940296

In [ ]:
row = lsdb.open_catalog(
    Path("../../internal/object_search/periodic_cand"),
    filters=[("diaObjectId", "==", OID)],
).compute().iloc[0]
row

In [ ]:
lc = row.lc.query(
    "~psfFlux_flag"
    " and ~pixelFlags_suspect"
    " and ~pixelFlags_saturated"
    " and ~pixelFlags_cr"
    " and ~pixelFlags_bad"
)
lc

In [ ]:
COLORS = {'u': '#0c71ff', 'g': '#49be61', 'r': '#c61c00',
          'i': '#ffc200', 'z': '#f341a2', 'y': '#5d0000'}

period = 2.0 * row.period_0

minimum_time = lc["helioMjd"][lc.query("band == 'r'")["psfMag"].idxmax()]
phase = (lc.helioMjd - minimum_time) % period / period
phase_grid = np.linspace(0, 1, 1000)

model = LombScargleMultiband()
model.fit(lc.midpointMjdTai, lc.psfMag, lc.psfMagErr, lc.band)

mean_model_mag = {
    # https://vizier.cds.unistra.fr/viz-bin/VizieR-5?-ref=VIZ68558c46e436&-out.add=.&-source=II/371/des_dr2&-c=059.129953%20-48.785229,eq=ICRS,rs=2&-out.orig=o
    "DECam.Y": 19.6831,
    # https://vizier.cds.unistra.fr/viz-bin/VizieR-5?-ref=VIZ685567dc55ef8&-out.add=.&-source=I/355/gaiadr3&-c=059.12995294496%20-48.78522838562,eq=ICRS,rs=2&-out.orig=o
    # Convert to AB mags, table 3 of https://www.aanda.org/articles/aa/pdf/2021/05/aa39587-20.pdf
    "G": 20.217580 + 25.8010 - 25.6874,
    "BP": 20.449993 + 25.1040 - 24.7479,
    "RP": 19.491499 + 25.3540 - 25.3385,
}
deviations = {
    "DECam.Y": 0.0186,
    "G": 0.008947,
    "BP": 0.131707,
    "RP": 0.070753,
}
model_std = {}
for band in 'ugrizy':
    idx = lc.band == band
    if np.sum(idx) == 0:
        continue
    plt.errorbar(
        phase[idx],
        lc.psfMag[idx],
        yerr=lc.psfMagErr[idx],
        fmt='o',
        color=COLORS[band],
        label=band,
    )
    mag_model = model.predict(phase_grid * period + minimum_time, band, period=0.5*period)
    mean_model_mag[band] = np.mean(mag_model)
    deviations[band] = np.std(lc.psfMag[idx] - model.predict(lc.psfMagErr[idx], band, period=period), ddof=1) / np.sqrt(np.sum(idx) - 1)
    model_std[band] = np.std(mag_model, ddof=1)
    plt.plot(
        phase_grid,
        mag_model,
        color=COLORS[band],
        linestyle='--',
        # label=f"{band} RR Lyr model",
    )
plt.gca().invert_yaxis()
plt.xlabel("Phase")
plt.ylabel("PSF mag")
plt.xlim([0, 1])
plt.title(f"diaObjectID {OID}\nRA=${row.ra:.5f}$, Dec=${row.dec:.5f}$, P=${period:.5f}$ d")
plt.legend(loc='lower right')
plt.savefig(f"{OID}.pdf")

In [ ]:
import numpy as np
from astropy import units as u

from fit_bb import black_body_magn

# Should be -27.05, table 3, https://iopscience.iop.org/article/10.3847/1538-4365/aabfdf/pdf
black_body_magn(5778*u.K, "i", solid_angle=np.pi*(np.arctan(1.0 * u.Rsun / u.au))**2)

In [ ]:
from fit_bb import fit_mags

BANDS = ['g', 'r', 'i', 'z', 'DECam.Y', 'G', 'BP', 'RP']

result = fit_mags(BANDS, [mean_model_mag[b] for b in BANDS], [deviations[b] for b in BANDS])
# Print the optimal values with errors, covariance matrix, square root of the sum of sqaured residuals
temperature, lg_solid_angle = result.x
residual_var = np.sum(result.fun ** 2) / (len(BANDS) - len(result.x))
cov = np.linalg.inv(result.jac.T @ result.jac) * residual_var
temperature_err, lg_solid_angle_err = np.sqrt(np.diag(cov))
print(f"Optimal temperature: {temperature:.2f} K ± {temperature_err:.2f} K")
print(f"Optimal log solid angle: {lg_solid_angle:.2f} log(steradian) ± {lg_solid_angle_err:.2f} log(steradian)")
print(f"Optimal solid angle: {10**lg_solid_angle:.2e} steradian")
print(f"Square root of the sum of squared residuals: {np.sqrt(result.cost):.2f}")
print("Covariance matrix:")
print(cov)

In [ ]:
# https://en.wikipedia.org/wiki/K-type_main-sequence_star#Spectral_standard_stars
r = 0.713 + (0.755 - 0.713) / (4830-4600) * (temperature-4600)
d = (r * u.R_sun) / (np.tan(np.sqrt(10**lg_solid_angle * u.steradian / np.pi)))
d_pc = d.to(u.pc)
d_rel_err = np.hypot(10**lg_solid_angle_err - 1, temperature_err / temperature)
print(f"Distance: {d_pc:.2f} ± {d_pc * d_rel_err:.2f}")